오차 역전파를 통해 오차를 기반으로 가중치를 수정한 후 더 좋은 성능을 낼 수 있도록 모형을 개선한다.

임의의 입력 값과 출력값의 설정

In [1]:
import numpy as np

In [2]:
input = np.array([[1,2,3]])
input.shape

(1, 3)

In [3]:
target = np.array([[0.5,0.1]])
target

array([[0.5, 0.1]])

In [4]:
print(input.shape, target.shape)

(1, 3) (1, 2)


임의의 가중치 설정

In [5]:
w1 = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])
w1.shape

(2, 3)

In [6]:
w2 = np.array([[0.7,0.9],[0.8,0.1]])
w2.shape

(2, 2)

In [7]:
b1 = np.array([[0.5],[0.5]])
b1.shape

(2, 1)

In [8]:
b2 = np.array([[0.3],[0.3]])
b2.shape

(2, 1)

순전파를 통한 출력값 계산

In [9]:
print(input.shape,w1.shape)

(1, 3) (2, 3)


In [10]:
g11 = w1@(input.T)+b1
g11

array([[2.7],
       [3.3]])

In [11]:
g12 = w1.dot(input.T)+b1
g12

array([[2.7],
       [3.3]])

In [12]:
print(np.equal(g11, g12))

[[ True]
 [ True]]


In [13]:
h = 1/(1+np.exp(-g11))
h

array([[0.93702664],
       [0.96442881]])

함수화

In [14]:
def hidden_cal(x,w,b):
  g = w.dot(x)+b
  h = 1/(1+np.exp(-g))
  return h

In [15]:
h2 = hidden_cal(h,w2,b2)
h2

array([[0.86103399],
       [0.75879129]])

비용 함수 정의 및 1차 미분식 계산

비용 함수로는 오차 제곱합을 사용, 미분식의 계산을 편리하게 하기 위한 1/2 연산의 추가

활성화 함수로 사용하는 시그모이드 함수의 미분

In [16]:
from sympy import Derivative, symbols

x = symbols('x')

In [17]:
fx = 1/(1+np.e**-x)

In [18]:
fprime = Derivative(fx, x).doit()
fprime

1.0*2.71828182845905**(-x)/(1 + 2.71828182845905**(-x))**2

역전파를 통한 1차 미분값 구하기

은닉층에서 출력층까지 연결되는 가중치가 변했을 때 비용 함수의 변화량

In [19]:
target.astype

<function ndarray.astype>

In [20]:
target[0,0]

0.5

In [21]:
target[0,1]

0.1

In [22]:
h2

array([[0.86103399],
       [0.75879129]])

In [23]:
h2[0]

array([0.86103399])

In [24]:
h2[1]

array([0.75879129])

In [25]:
h2[0]-target[0,0]

array([0.36103399])

In [26]:
h2[0]*(1-h2[0])

array([0.11965446])

거꾸러 올라가는 과정의 수식 계산

In [27]:
w2 # 은닉층의 가중치 행렬의 전치

array([[0.7, 0.9],
       [0.8, 0.1]])

In [28]:
h2 # 은닉층의 입력값

array([[0.86103399],
       [0.75879129]])

In [29]:
b2 # 은닉층의 편향 가중치

array([[0.3],
       [0.3]])

In [30]:
print(w2.shape, h2.shape)

(2, 2) (2, 1)


In [31]:
w2@(h2)+b2

array([[1.58563595],
       [1.06470632]])

In [32]:
print(w2[0,0], h2[0,0])

0.7 0.8610339864424298


In [33]:
w2[0,0]*h2[0,0]

0.6027237905097008

In [34]:
w2[0,1]*h2[1,0]

0.6829121630087827

In [35]:
w2[0,0]*h2[0,0]+w2[0,1]*h2[1,0]+b2[0]

array([1.58563595])

위 수식을 미분한 값을 얻기 위한 함수 작성

행렬곱을 펼친 수식을 얻어야 한다.


In [36]:
x = symbols('x')
y = symbols('y')

b = x+y
c = b+y
c

x + 2*y

In [53]:
def propagtaion_diff(w, h, b):
  """
  Arguments:
    w : 가중치 행렬의 전치
    h : 입력 데이터
    b : 가중치 편향값
  """
  num_unit = w.shape[1] # 은닉층의 unit 개수
  num_output = w.shape[0] # 출력층의 unit 개수, bias의 개수
  for i in range(num_unit):
    exec(f"b{i}=symbols('b{i}')")
    exec(f"h{i}{0}=symbols('h{i}{0}')")
    for j in range(num_output):
      exec(f"w{i}{j}=symbols('w{i}{j}')")
  

  list_num = []
  z1 = 0

  for i in range(num_unit):
    z1 = eval(f"b{i}")    
    for j in range(num_output):
      z1 += eval(f"w{i}{j}*h{j}{0}")
    
    list_num.append(z1)
    z1 = 0

  return list_num

In [54]:
list_num = propagtaion_diff(w2, h2, b2)
list_num

[b0 + h00*w00 + h10*w01, b1 + h00*w10 + h10*w11]

In [39]:
list_num[0]

array([[h00*w00 + h10*w01 + 0.3],
       [h00*w00 + h10*w01 + 0.3]], dtype=object)

In [40]:
print(len(w2), len(h2[0]), len(w2[0]))

2 1 2


In [41]:
w2[0,0]

0.7

In [42]:
bw41 = (h2[0]-target[0,0]) * (h2[0]*(1-h2[0])) * h[0]
bw41

array([0.04047892])

In [43]:
bw42 = (h2[1]-target[0,1]) * (h2[1]*(1-h2[1])) * h[0]
bw42

array([0.11298352])

In [44]:
bw51 = (h2[0]-target[0,0]) * (h2[0]*(1-h2[0])) * h[1]
bw51

array([0.04166268])

In [45]:
bw52 = (h2[1]-target[0,1]) * (h2[1]*(1-h2[1])) * h[1]
bw52

array([0.11628758])

In [46]:
bb2 = (h2[0]-target[0,0]) * (h2[0]*(1-h2[0])) * 1 + (h2[1]-target[0,1]) * (h2[1]*(1-h2[1])) * 1
bb2

array([0.16377596])

은닉층에서 입력층으로 거슬러 올라가는 계산